
# Practicum II - Movie Database Applicaton
### Collaborators: Abasamia Akpan, Rebecca Brent and Alessia Pizzoccheri

This Notebook provides context, documentation and sample code for a Database Application using IMBd datasets; for clarity, this document has been divided into the following sections:

### 1. Introduction
### 2. Assumptions
### 3. Logical Models
### 4. Database and Relations Definition¶
### 1. Introduction
#### 1.1. Looking at the data
In this section, we analyze and deconstruct the various data sets found on the IMBd page; data is organized in seven TSV files, specifically:

Title AKAs
Title Basics
Title Episodes
Title Ratings
Title Crew
Name Basics
Title Principles Our first step is to establish relationships between each data set and identify a primary key (PK) on each table. Since no information was provided regarding the role of columns, we devised a solution to find unique identifiers within each data set; that is, for each table we compare the total number of rows against the total number of unique values for a given column.
1.2 Establishing PKs
Following the system aforementioned, we identified the following primary keys

## Notes


#### title_basics is fine but wont let me add a null key at the end to store the foreign keys in the other table that dont map to a primary key in title_basics table so i commented it out.  Loads 99 except for that%

#### title ratings has foreign keys that dont exist . I tried to mask them and they worked before but table insertion failed today- #### We could make a decision if we want to discard foreign keys tconst, parentTconst and nconst that dont map to a primary key in title_basics or name_basics table and consider them as bad data

#### title episodes has the same issue as above

#### title AKA's has the same issue as above

#### name basics is fine

#### principals is fine 

#### all the other tables should be working

#### we should probably chage some lengths of some columns  or do this if performance is a concern
#### https://stackoverflow.com/questions/8746207/1071-specified-key-was-too-long-max-key-length-is-1000-bytes
Have no idea how to implement though
.....

In [50]:

import pandas as pd
import numpy as np
import os
import sys
import pip
pip.main(['install','prettytable'])
pip.main(['install','mysql-connector-python'])
pip.main(['install','tqdm'])
pip.main(['install','pymysql'])

from tqdm import tqdm



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [149]:
import mysql.connector
from prettytable import PrettyTable 
from prettytable import from_csv
from mysql.connector import Error

def create_connection_nodb(host_name, user_name, user_password):
    """ create a connection to mysql host
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_database(connection, db_name):
    """ create a database 
    :param connection: Connection object
    :param db_name: name of database in form of string
    :return:
    """
    cursor = connection.cursor()
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + db_name
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(host_name, user_name, user_password, db_name):
    """ create a connection to the (possibly newly created) database
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :param db_name: string name of database 
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        table_name = create_table_sql.split(" ")[5].split("(")[0]
        print("Successfully created table: " + table_name)
    except Error as e:
        print(e)
        
def main():
    
    database = "imbd_application"
    host_name = "localhost"
    user_name = "root"
    user_password = "Odyssey22"
    
    create_table_titlebasics = """CREATE TABLE IF NOT EXISTS titlebasics(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    titleType TEXT,
                                    primaryTitle TEXT,
                                    originalTitle TEXT,
                                    isAdult BOOLEAN,
                                    startYear DATE,
                                    endYear DATE,
                                    runTimeMinutes TIME)"""
    
    create_table_titleratings = """CREATE TABLE IF NOT EXISTS titleratings(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    averageRating FLOAT,
                                    numVotes INT,
                                    FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""


    create_table_titleepisodes = """CREATE TABLE IF NOT EXISTS titleepisodes(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    seasonNumber INT,
                                    episodeNumber INT,
                                    parentTconst CHAR(9),
                                    FOREIGN KEY (parentTconst) REFERENCES titlebasics(tconst))"""


    create_table_namebasics = """CREATE TABLE IF NOT EXISTS namebasics(
                                nconst VARCHAR(11) PRIMARY KEY,
                                primaryname TEXT,
                                birthyear DATE,
                                deathyear DATE,
                                noofmovies INT,
                                age INT,
                                currentdate DATE)"""
    
    
    create_table_genres = """CREATE TABLE IF NOT EXISTS genres(
                                genreID INT AUTO_INCREMENT  PRIMARY KEY,
                                genre VARCHAR(256) UNIQUE)"""
    
    create_table_primaryprofession = """CREATE TABLE IF NOT EXISTS primaryprofession(
                                primaryprofessionID INT AUTO_INCREMENT PRIMARY KEY,
                                profession VARCHAR(256) UNIQUE)"""
    
 
    
    
    create_table_genretitlebasics = """CREATE TABLE IF NOT EXISTS genretitlebasics(
                                titlebasicsID VARCHAR(11),
                                genreID INT,
                                PRIMARY KEY (titlebasicsID, genreID),
                                FOREIGN KEY (titlebasicsID) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (genreID) REFERENCES genres(genreID))"""
    

    
    create_table_primaryprofessiontitlenames = """CREATE TABLE IF NOT EXISTS primaryprofessiontitlenames(
                                primaryprofessionid INT,
                                namebasicsid VARCHAR(11),
                                PRIMARY KEY (primaryprofessionid, namebasicsid),
                                FOREIGN KEY (primaryprofessionid) REFERENCES primaryprofession(primaryprofessionid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst))"""
    
    create_table_knownfortitles = """CREATE TABLE IF NOT EXISTS knownfortitlenames(
                                namebasicsid VARCHAR(11),
                                knownfortitles VARCHAR(11),
                                PRIMARY KEY (namebasicsid, knownfortitles),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst),
                                FOREIGN KEY (knownfortitles) REFERENCES titlebasics(tconst))"""
    
    
    conn = create_connection_nodb(host_name, user_name, user_password)
    
    # create database object if not already created and then connect the cursor to it
    if conn is not None:
        c = conn.cursor()
        c.execute('DROP DATABASE imbd_application')
        create_database(conn, database)
        conn = create_connection(host_name, user_name, user_password, database)
        
        # create tables
        if conn is not None:
            
            # create Title Basics table
            create_table(conn, create_table_titlebasics)

            # create Title Ratings table
            create_table(conn, create_table_titleratings)
            
            # create title episodes table
            create_table(conn, create_table_titleepisodes)
            
            # create name basics table
            create_table(conn, create_table_namebasics)
            
            # create genres table
            create_table(conn,create_table_genres)
            
            # create known for titles table
            create_table(conn,create_table_knownfortitles)
            
            # create primary profession table
            create_table(conn,create_table_primaryprofession)
            
            # create genre title basics table
            create_table(conn, create_table_genretitlebasics)

            # create Primary Profession Title Names table
            create_table(conn, create_table_primaryprofessiontitlenames)
            
                
        else:
            print("Error! cannot create the database connection.")
    
    else:
        print("Error! Cannot create the MySQL connection.")
    
    


if __name__ == '__main__':
    main()


Connection to MySQL DB successful
Database created successfully
Connection to MySQL DB successful
Successfully created table: titlebasics
Successfully created table: titleratings
Successfully created table: titleepisodes
Successfully created table: namebasics
Successfully created table: genres
Successfully created table: knownfortitlenames
Successfully created table: primaryprofession
Successfully created table: genretitlebasics
Successfully created table: primaryprofessiontitlenames


## SQL communication helper ##

In [81]:
import sqlalchemy
database_username = 'root'
database_password = 'Odyssey22'
database_ip       = ''
database_name     = 'imbd_application'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [82]:
## Helper Functions

In [83]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
    return 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [84]:
def preprocess_dataframe_parsing(data_frame, column_name, final_column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    data_preprocessed_unique= pd.DataFrame(data_frame.name.unique())
    data_preprocessed_unique_column_rename=data_preprocessed_unique.rename(columns={0:final_column_name})
    
    return data_preprocessed_unique_column_rename

## Title Basics

In [85]:
# read title basic file
title_basics = pd.read_csv('title.basics.tsv', delimiter='\t',encoding='utf-8')

c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [150]:
def preprocess_dataframe__for_genres(data_frame):
    df = data_frame
    lst_col = 'genres'
  
    df = df.drop(['titleType'], axis=1)
    df = df.drop(['primaryTitle'], axis=1)
    df = df.drop(['originalTitle'], axis=1)
    df=df.drop(['isAdult'], axis=1)
    df=df.drop(['startYear'], axis=1)
    df=df.drop(['endYear'], axis=1)
    df=df.drop(['runtimeMinutes'],axis=1)
    
    x = df.assign(**{lst_col:df[lst_col].str.split(',')})
    genres_df = x.explode('genres')

    return genres_df

df =  preprocess_dataframe__for_genres(title_basics)


genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres", "genre")
genre_dict = genres_preprocessed['genre'].to_dict()
inv_map = {v: k for k, v in genre_dict.items()}
df = df.replace({"genres":inv_map})
df = df.rename(columns={"tconst" : "titlebasicsID", "genres" : "genreID"})
df.genreID = df.genreID + 1 

In [151]:
df

,titlebasicsID,genreID
0,tt0000001,1
0,tt0000001,2
1,tt0000002,3
1,tt0000002,2
2,tt0000003,3
...,...,...
7316919,tt9916852,17
7316920,tt9916856,2
7316921,tt9916880,3
7316921,tt9916880,4


In [152]:
batch_size=100000
length=(len(genres_preprocessed)//batch_size)+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.64it/s]


In [153]:
def preprocess_dataframe_title_basics(data_frame):
    title_basics_new_series=data_frame

    b = pd.to_datetime(title_basics_new_series['runtimeMinutes'], errors='coerce', format='%M')

    b.to_frame().runtimeMinutes
    df_1 = pd.DataFrame(dict(time_stamps = b)) 

    df_1['runtimeMinutes'] = df_1['time_stamps'].dt.minute
    title_basics_new_series=title_basics_new_series.drop(['runtimeMinutes'],axis=1)
    title_basics_new_series['runtimeMinutes']=df_1['runtimeMinutes']
    
    title_basics_new_frame_transpose=title_basics_new_series.astype({'isAdult': 'int32'})
    
    title_basics_new_frame_transpose["isAdult"]=title_basics_new_frame_transpose.isAdult.mask(title_basics_new_frame_transpose.isAdult > 1,1)

    title_basics_new_frame_transpose['startYear']= pd.to_datetime(title_basics.startYear, errors='coerce', format='%Y')

    
    title_basics_new_frame_transpose['endYear']= pd.to_datetime(title_basics.endYear, errors='coerce', format='%Y')
    title_basics_new_frame_transpose=title_basics_new_frame_transpose.drop(['genres'],axis=1)
  

    title_basics_new_frame_transpose_new=title_basics_new_frame_transpose.rename(columns={"runtimeMinutes": "runTimeMinutes"})

    return title_basics_new_frame_transpose_new

title_basics_preprocessed=preprocess_dataframe_title_basics(title_basics)

In [154]:
batch_size = 10000
length = len(title_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_basics_preprocessed, batch_size),total=length):
    load_dataframe(chunk,'titlebasics')

100%|████████████████████████████████████████████████████████████████████████████████| 732/732 [05:02<00:00,  2.42it/s]


In [155]:
batch_size = 10000
length = len(df)//batch_size+1

for chunk in tqdm(batch(df,batch_size),total=length):
    load_dataframe(chunk,'genretitlebasics')
    
del df

100%|██████████████████████████████████████████████████████████████████████████████| 1163/1163 [07:55<00:00,  2.44it/s]


## Title ratings ##

In [36]:
# read data from title ratings file
title_ratings = pd.read_csv('/Users/abasiamaakpan/Downloads/title.ratings.tsv', delimiter='\t',encoding='utf-8')
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1656
1,tt0000002,6.1,201
2,tt0000003,6.5,1369
3,tt0000004,6.2,122
4,tt0000005,6.2,2151
...,...,...,...
1086697,tt9916580,7.2,5
1086698,tt9916690,6.6,5
1086699,tt9916720,6.0,65
1086700,tt9916766,6.9,14


In [469]:
def preprocess_dataframe_title_ratings(data_frame):
    title_ratings_new_series=data_frame
    
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'averageRating': 'float'})
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'numVotes': 'int32'})
    #title_ratings_preprocessed.tconst = title_ratings_preprocessed.tconst.mask(~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)
    return title_ratings_new_frame_avg_rating
title_ratings_preprocessed=preprocess_dataframe_title_ratings(title_ratings)

In [458]:
len(title_ratings_preprocessed)

1086702

In [464]:
#title_ratings_preprocessed[~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst)]

title_ratings_preprocessed.tconst = title_ratings_preprocessed.tconst.mask(~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)



,tconst,averageRating,numVotes


In [ ]:
batch_size = 100000
length = len(title_ratings_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_ratings_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleratings')

## Title Episodes ##

In [42]:
# read title episodes data
title_episodes = pd.read_csv('/Users/abasiamaakpan/Downloads/title.episode.tsv', delimiter='\t',encoding='utf-8')
title_episodes

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
...,...,...,...,...
5252722,tt9916846,tt1289683,3,18
5252723,tt9916848,tt1289683,3,17
5252724,tt9916850,tt1289683,3,19
5252725,tt9916852,tt1289683,3,20


In [490]:
def preprocess_dataframe_title_episodes(data_frame):
    title_episodes_new_series=data_frame
    title_episodes_coerce_seasonNum = pd.to_numeric(title_episodes_new_series['seasonNumber'], errors='coerce')
    title_episodes_coerce_episodeNum = pd.to_numeric(title_episodes_new_series['episodeNumber'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_episodes_coerce_seasonNum)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_episodes_coerce_episodeNum)) 

    df_1['seasonNumber'] = df_1['test_column_1']
    df_2['episodeNumber'] = df_2['test_column_2']
    
    title_episodes_new_frame=title_episodes_new_series.drop(['seasonNumber'],axis=1)
    title_episodes_new_frame=title_episodes_new_series.drop(['episodeNumber'],axis=1)
    
    title_episodes_new_frame['seasonNumber']=df_1['seasonNumber']
    title_episodes_new_frame['episodeNumber']=df_2['episodeNumber']
    return title_episodes_preprocessed

title_episodes_preprocessed = preprocess_dataframe_title_episodes(title_episodes)
title_episodes_preprocessed

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0
...,...,...,...,...
5252722,tt9916846,tt1289683,3.0,18.0
5252723,tt9916848,tt1289683,3.0,17.0
5252724,tt9916850,tt1289683,3.0,19.0
5252725,tt9916852,tt1289683,3.0,20.0


In [491]:
title_episodes_preprocessed[~title_episodes_preprocessed.parentTconst.isin(title_basics_preprocessed.tconst)]



,tconst,parentTconst,seasonNumber,episodeNumber


In [492]:
title_episodes_preprocessed.tconst = title_episodes_preprocessed.tconst.mask(~title_episodes_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)



In [ ]:
title_episodes_preprocessed[~title_episodes_preprocessed.parentTconst.isin(title_basics_preprocessed.tconst)]



In [ ]:
batch_size = 100000
length = len(title_episodes_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_episodes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleepisodes')

## Name Basics

In [116]:
name_basics = pd.read_csv('name.basics.tsv', delimiter='\t',encoding='utf-8')
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0037382,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0059956,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0077975,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0050976,tt0083922,tt0050986"
...,...,...,...,...,...,...
10484335,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10484336,nm9993716,Essias Loberg,\N,\N,NaN,\N
10484337,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10484338,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [156]:
def preprocess_dataframe_name_basics(data_frame):
    name_basics_new_series=data_frame
    name_basics_new_series=name_basics_new_series.drop(['primaryProfession', 'knownForTitles'],axis=1)
    
    name_basics_new_series['birthYear']= pd.to_datetime(name_basics_new_series.birthYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]
    
    name_basics_new_series['deathYear']= pd.to_datetime(name_basics_new_series.deathYear, errors='coerce', format='%Y')
    
    return name_basics_new_series

name_basics_preprocessed = preprocess_dataframe_name_basics(name_basics)

name_basics_preprocessed["noofmovies"] = ""
name_basics_preprocessed["age"] = ""
name_basics_preprocessed["currentdate"] = ""

In [159]:
def preprocess_dataframe_for_primaryprof(data_frame):
    df = data_frame
    lst_col = 'primaryProfession'
  
    df = df.drop(['primaryName'], axis=1)
    df = df.drop(['birthYear'], axis=1)
    df = df.drop(['deathYear'], axis=1)
    df=df.drop(['knownForTitles'], axis=1)

    
    x = df.assign(**{lst_col:df[lst_col].str.split(',')})
    genres_df = x.explode('primaryProfession')

    return genres_df

df =  preprocess_dataframe_for_primaryprof(name_basics)


profession_preprocessed=preprocess_dataframe_parsing(name_basics.primaryProfession, "primaryProfession", "profession")
prof_dict = profession_preprocessed['profession'].to_dict()
inv_map = {v: k for k, v in prof_dict.items()}
df = df.replace({"primaryProfession":inv_map})
df = df.rename(columns={"nconst" : "namebasicsID", "primaryProfession" : "primaryprofessionID"})
df.primaryprofessionID = df.primaryprofessionID + 1

In [160]:
name_basics_preprocessed_new= pd.to_numeric(name_basics_preprocessed['noofmovies'], errors='coerce')
name_basics_preprocessed_new = pd.to_numeric(name_basics_preprocessed['age'], errors='coerce')
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed.currentdate, errors='coerce', format='%Y')


In [161]:
df_1 = pd.DataFrame(dict(test_column_1 = name_basics_preprocessed_new)) 
df_2 = pd.DataFrame(dict(test_column_2 = name_basics_preprocessed_new)) 

df_1['noofmovies'] = df_1['test_column_1']
df_2['age'] = df_2['test_column_2']

name_basics_preprocessed['noofmovies']=df_1['noofmovies']
name_basics_preprocessed['age']=df_2['age']
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed['currentdate'])

In [162]:
batch_size = 10000
length = len(name_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

100%|██████████████████████████████████████████████████████████████████████████████| 1049/1049 [05:06<00:00,  3.43it/s]


In [163]:
batch_size=10000
length=len(profession_preprocessed)//batch_size+1
for chunk in tqdm(batch(profession_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'primaryprofession')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.42it/s]


In [164]:
batch_size = 10000
length = (len(df)//batch_size)+1

for chunk in tqdm(batch(df,batch_size),total=length):
    load_dataframe(chunk,'primaryprofessiontitlenames')

100%|██████████████████████████████████████████████████████████████████████████████| 1363/1363 [09:22<00:00,  2.42it/s]


In [165]:
def preprocess_dataframe_for_knownfortitles(data_frame):
    df = data_frame
    lst_col = 'knownForTitles'
  
    df = df.drop(['primaryName'], axis=1)
    df = df.drop(['birthYear'], axis=1)
    df = df.drop(['deathYear'], axis=1)
    df=df.drop(['primaryProfession'], axis=1)
    
    x = df.assign(**{lst_col:df[lst_col].str.split(',')})
    kft_df = pd.DataFrame({
                      col:np.repeat(x[col].values, x[lst_col].str.len())
                      for col in x.columns.difference([lst_col])
                      }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

    return kft_df

kft_df =  preprocess_dataframe_for_knownfortitles(name_basics)


kft_df = kft_df[kft_df.knownForTitles != '\\N']
kft_df = kft_df.rename(columns={"nconst" : "namebasicsID", "knownFortitles" : "knownfortitlesID"})

AttributeError: 'DataFrame' object has no attribute 'knownfortitlesID'

In [166]:
kft_df

,namebasicsID,knownForTitles
0,nm0000001,tt0031983
1,nm0000001,tt0072308
2,nm0000001,tt0050419
3,nm0000001,tt0053137
4,nm0000002,tt0117057
...,...,...
18337840,nm9993713,tt8325250
18337841,nm9993713,tt10449366
18337842,nm9993713,tt10709066
18337843,nm9993714,tt2455546


In [167]:
batch_size=100000
length=len(kft_df)//batch_size+1
for chunk in tqdm(batch(kft_df,batch_size),total=length):
    load_dataframe(chunk,'knownFortitles')

  0%|                                                                                          | 0/165 [00:00<?, ?it/s]c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\pandas\io\sql.py:1423: UserWarning: The provided table name 'knownFortitles' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
100%|████████████████████████████████████████████████████████████████████████████████| 165/165 [04:28<00:00,  1.63s/it]


## Question 5

In [ ]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

operation = """SELECT COUNT(KTN.knownfortitlesID)
               FROM namebasics AS NB, knownfortitlenames AS KTN
               WHERE KTN.knownfortitlesID IN (SELECT KT.knownfortitlesID 
                                              FROM knownforTitles AS KT 
                                              WHERE KT.titles IN (SELECT tconst
                                                                  FROM titlebasics AS TB 
                                                                  WHERE TB.titletype = 'Movie'))
                GROUP BY namebasicsID
                LIMIT 5"""

for result in c.execute(operation, multi=True):
    if result.with_rows:
        print("Rows produced by statement '{}':".format(
          result.statement))
        print(result.fetchall())
    else:
        print("Number of rows affected by statement '{}': {}".format(
          result.statement, result.rowcount))